# 使用 Stable Diffusion 修改 DeepRacer 赛道图像并分析模型的行为

在本实验室练习中，我们将使用修改后的 DeepRacer 赛道图像来分析我们的 DeepRacer 模型在现实世界中可能的表现。

我们的起始图像来源于用于训练和评估 DeepRacer 模型的虚拟世界 DeepRacer 控制台。使用 Stable Diffusion，我们修改这些图像以模拟“现实世界”的赛道，通过增加照明条件和赛道背景的变化。

为了节约 GPU 资源，我们在这里不部署 Stable Diffusion 模型。如果您对如何自己部署这些模型感兴趣，请参阅 `01_stablediffusion_original.ipynb`，其中包含了完整的原始代码。

## 架构

下面是用来生成我们将用来测试模型的“真实感”赛道图像的架构。

![](./images/lab2_arch.png)

# 使用我们更新的DeepRacer图像来测试模型性能

使用 `output_images` 中的图像，让我们测试其中一个DeepRacer模型并查看其性能表现。

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import boto3
import logging
import shutil
import os
import glob
import math
import tarfile
import requests
import json
%matplotlib inline

In [ ]:
!pip install shapely
!pip install opencv-python-headless

In [ ]:
# Shapely Library
from shapely.geometry import Point, Polygon
from shapely.geometry.polygon import LinearRing, LineString

In [ ]:
from log_analysis import *
from os import listdir
from os.path import isfile, join

In [ ]:
# 设置模型名称（注意，目前我们只支持仓库中包含的演示模型）
model_name="AtoZ-CCW-Centerline"

In [ ]:
!rm -rf ./intermediate_checkpoint

接下来，我们将模型复制到 `02_stablediffusion` 内的正确路径中。

In [ ]:
!mkdir -p intermediate_checkpoint/model-artifacts/

if model_name == "AtoZ-CCW-Centerline":
    print("Using AtoZ-CCW-Centerline demo model")
    !cp -R ../deepracer_models/AtoZ-CCW-Centerline/ intermediate_checkpoint/model-artifacts/
elif model_name == "AtoZ-CCW-Steering-Penalty":
    print("Using AtoZ-CCW-Steering-Penalty demo model")
    !cp -R ../deepracer_models/AtoZ-CCW-Steering-Penalty/ intermediate_checkpoint/model-artifacts/
else:
    print("Check your model name...it doesn't appear to be one of the demo models.")

接下来，让我们读取模型的元数据和动作空间变量。这些将在未来的步骤中用于渲染图像的热图。

In [ ]:
with open("intermediate_checkpoint/model-artifacts/{}/model_metadata.json".format(model_name),"r") as jsonin:
    model_metadata=json.load(jsonin)
sensor = [sensor for sensor in model_metadata['sensor'] if sensor != "LIDAR"][0]
model_metadata

In [ ]:
# Track Segment Labels
action_names = []
for action in model_metadata['action_space']:
    action_names.append("ST"+str(action['steering_angle'])+" SP"+"%.2f"%action["speed"])
action_names

在下一步中，我们读取在此实验室创建的图像并将其存储在一个数组中。

In [ ]:
import glob
img_path = "output_images"
all_files = sorted(glob.glob(img_path + '/*.png'))

我们将使用 TensorFlow 来运行模型以处理新创建的图像。让我们安装它。

In [ ]:
!pip uninstall numpy -y
!pip install --quiet tensorflow

在接下来的两个步骤中，我们将导入以 protobuf 格式存储的模型图定义，并将新图像输入到模型中。

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
%%capture tensor_setup_output
import logging
import tensorflow.compat.v1 as tf
from tensorflow.python.platform import gfile
from PIL import Image
tf.disable_v2_behavior()

logger = tf.get_logger()
logger.setLevel(logging.ERROR)


GRAPH_PB_PATH = 'intermediate_checkpoint/'

def load_session(pb_path):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                    log_device_placement=True))
    print("load graph:", pb_path)
    with gfile.FastGFile(pb_path,'rb') as f:
        graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    names = []
    for t in graph_nodes:
        names.append(t.name)

    # For front cameras/stereo camera use the below
    x = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_0/{}/{}:0'.format(sensor, sensor))
    y = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_1/ppo_head_0/policy:0')

    return sess, x, y

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [ ]:
print(GRAPH_PB_PATH)

In [ ]:
%%capture tensorflow_logs
import logging
model_inference = []
iterations = [7,8,9]
models_file_path = glob.glob("{}model-artifacts/{}/model/model*.pb".format(GRAPH_PB_PATH, model_name))
for model_file in models_file_path:

    model, obs, model_out = load_session(model_file)
    arr = []
    for f in all_files[:]:
        img = Image.open(f)
        img_arr = np.array(img)
        img_arr = rgb2gray(img_arr)
        img_arr = np.expand_dims(img_arr, axis=2)
        current_state = {"observation": img_arr} #(1, 120, 160, 1)
        y_output = model.run(model_out, feed_dict={obs:[img_arr]})[0]
        arr.append (y_output)
    model_inference.append(arr)
    model.close()
    tf.reset_default_graph()

我们将在预训练的 DeepRacer 模型上使用 OpenCV 方法处理 Stable Diffusion 生成的图像，并将其叠加到热图上，同时考虑 DeepRacer 模型的权重。

In [ ]:
import cv2

def visualize_gradcam_discrete_ppo(sess, rgb_img, category_index=0, num_of_actions=5):
    '''
    @inp: model session, RGB Image - np array, action_index, total number of actions
    @return: overlayed heatmap
    '''

    img_arr = np.array(img)
    img_arr = rgb2gray(img_arr)
    img_arr = np.expand_dims(img_arr, axis=2)

    x = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_0/{}/{}:0'.format(sensor, sensor))
    y = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_1/ppo_head_0/policy:0')
    feed_dict = {x:[img_arr]}

    #Get the policy head for clipped ppo in coach
    model_out_layer = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_1/ppo_head_0/policy:0')
    loss = tf.multiply(model_out_layer, tf.one_hot([category_index], num_of_actions))
    reduced_loss = tf.reduce_sum(loss[0])

    # For front cameras use the below
    conv_output = sess.graph.get_tensor_by_name('main_level/agent/main/online/network_1/{}/Conv2d_4/Conv2D:0'.format(sensor))

    grads = tf.gradients(reduced_loss, conv_output)[0]
    output, grads_val = sess.run([conv_output, grads], feed_dict=feed_dict)
    weights = np.mean(grads_val, axis=(1, 2))
    cams = np.sum(weights * output, axis=3)

    ##im_h, im_w = 120, 160##
    im_h, im_w = rgb_img.shape[:2]

    cam = cams[0] #img 0
    image = np.uint8(rgb_img[:, :, ::-1] * 255.0) # RGB -> BGR
    cam = cv2.resize(cam, (im_w, im_h)) # zoom heatmap
    cam = np.maximum(cam, 0) # relu clip
    heatmap = cam / np.max(cam) # normalize
    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET) # grayscale to color
    cam = np.float32(cam) + np.float32(image) # overlay heatmap
    cam = 255 * cam / (np.max(cam) + 1E-5) ##  Add expsilon for stability
    cam = np.uint8(cam)[:, :, ::-1] # to RGB

    return cam

现在，让我们循环遍历输出文件夹中 Stable Diffusion 生成的图像，并将其传递给上面定义的热图可视化函数。生成的热图将存储到名为 "heatmaps" 的数组中。

In [ ]:
%%capture heatmap_cell_logs
model_path = models_file_path[0] #Change this to your model 'pb' frozen graph file

model, obs, model_out = load_session(model_path)
heatmaps = []
print(all_files)
#Just need to match up the shape of the neural network
if 'action_space_type' in model_metadata and model_metadata['action_space_type']=='continuous':
    num_of_actions=2
else:
    num_of_actions=len(action_names)

for f in all_files[:6]:
    img = np.array(Image.open(f))
    heatmap = visualize_gradcam_discrete_ppo(model, img, category_index=0, num_of_actions=num_of_actions)
    heatmaps.append(heatmap)
tf.reset_default_graph()

In [ ]:
# Display the images used to generate the heatmaps
image_folder = 'output_images'

# List all files in the directory
files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))]

# Display all images
for f in all_files[:6]:
    img = Image.open(f)
    plt.figure(figsize=(6, 6))  # Adjust the figure size as needed
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.title(os.path.basename(f))  # Optional: add title with file name
    plt.show()

In [ ]:
for i in range(len(heatmaps)):
    plt.imshow(heatmaps[i])
    plt.show()

## 实验结论

在这次实验中，我们已经成功地：

- 将更真实的赛道图像输入到我们的DeepRacer模型中
- 生成热图以了解我们的模型关注输入图像的哪些部分

在这里，我们已经看到了生成式AI模型如何帮助测试和评估各种系统，包括其他机器学习模型。